In [74]:
import tensorflow as tf
import keras
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Lambda, Flatten
from keras import backend as K

**BASIC FUCKING TEST FOR FUCKING SANITY**

In [81]:
def max_function(x):
    return K.max(x, axis=1)

input_tensor = Input(shape=(5,5,1))
identity = Lambda(max_function)(input_tensor)
m = Model(inputs=input_tensor, outputs=identity)
m.predict(np.array([[[[1],[2],[3],[4],[5]],
                    [[1],[2],[3],[4],[5]],
                    [[1],[2],[3],[4],[5]],
                    [[1],[2],[3],[4],[5]],
                    [[1],[2],[3],[4],[5]]
                   ]]))

array([[[ 1.],
        [ 2.],
        [ 3.],
        [ 4.],
        [ 5.]]], dtype=float32)

**Test inputs**

In [82]:
batch_size = 2
dim = 5
input_shape = (dim,dim,1)
batch_shape = (batch_size,) + input_shape
test_tensor = Input(shape=input_shape)
label_tensor = Input(shape=input_shape)
test_in = np.empty(batch_shape)

for i in range(dim):
    for j in range(dim):
        test_in[:,i,j,:] = i + j
        
positive_labels = np.ones(batch_shape)
negative_labels = np.full(batch_shape, -1)
diagonal_negative_labels = np.ones(batch_shape)

for i in range(dim):
    diagonal_negative_labels[:,i,i,:] = -1

**K.max()**

In [84]:
max_col = Lambda(max_function)(test_tensor)
max_row = Lambda(max_function)(max_col)
model = Model(inputs=test_tensor, outputs=max_row)
print(test_in.shape)
model.predict(test_in)

(2, 5, 5, 1)


array([[ 8.],
       [ 8.]], dtype=float32)

**Subtract**

In [88]:
def sub_function(x):
    max_scores = K.max(K.max(x, axis=1, keepdims=True), axis=2, keepdims=True)
    return x - max_scores

max_zero = Lambda(sub_function)(test_tensor)
model = Model(inputs=test_tensor, outputs=max_zero)
model.predict(test_in)

array([[[[-8.],
         [-7.],
         [-6.],
         [-5.],
         [-4.]],

        [[-7.],
         [-6.],
         [-5.],
         [-4.],
         [-3.]],

        [[-6.],
         [-5.],
         [-4.],
         [-3.],
         [-2.]],

        [[-5.],
         [-4.],
         [-3.],
         [-2.],
         [-1.]],

        [[-4.],
         [-3.],
         [-2.],
         [-1.],
         [ 0.]]],


       [[[-8.],
         [-7.],
         [-6.],
         [-5.],
         [-4.]],

        [[-7.],
         [-6.],
         [-5.],
         [-4.],
         [-3.]],

        [[-6.],
         [-5.],
         [-4.],
         [-3.],
         [-2.]],

        [[-5.],
         [-4.],
         [-3.],
         [-2.],
         [-1.]],

        [[-4.],
         [-3.],
         [-2.],
         [-1.],
         [ 0.]]]], dtype=float32)

**logarithm**

In [96]:
keras.layers.merge

def log_loss(x):
    stable = sub_function(x)
    probs = K.exp(stable) + 1
    return K.log(probs)

def avg_loss(x):
    return K.mean(x)

custom_loss = Lambda(log_loss)(test_tensor)
avg_loss = Lambda(avg_loss)(custom_loss)

model = Model(inputs=test_tensor, outputs=avg_loss)
model.predict(test_in)

array([ 0.08088002,  0.08088002], dtype=float32)

**Multiply**

In [99]:
def product_fn(args):
    return args[0] * args[1]

product = Lambda(product_fn)([test_tensor, label_tensor])
model = Model(inputs=[test_tensor, label_tensor], outputs=product)
model.predict([test_in, negative_labels])

array([[[[-0.],
         [-1.],
         [-2.],
         [-3.],
         [-4.]],

        [[-1.],
         [-2.],
         [-3.],
         [-4.],
         [-5.]],

        [[-2.],
         [-3.],
         [-4.],
         [-5.],
         [-6.]],

        [[-3.],
         [-4.],
         [-5.],
         [-6.],
         [-7.]],

        [[-4.],
         [-5.],
         [-6.],
         [-7.],
         [-8.]]],


       [[[-0.],
         [-1.],
         [-2.],
         [-3.],
         [-4.]],

        [[-1.],
         [-2.],
         [-3.],
         [-4.],
         [-5.]],

        [[-2.],
         [-3.],
         [-4.],
         [-5.],
         [-6.]],

        [[-3.],
         [-4.],
         [-5.],
         [-6.],
         [-7.]],

        [[-4.],
         [-5.],
         [-6.],
         [-7.],
         [-8.]]]], dtype=float32)

**Computing with labels**

In [103]:
def loss_fn(inputs):
    y_true, y_pred = inputs
    product = y_true * y_pred
    max_prod = K.max(K.max(product, axis=1, keepdims=True), axis=2, keepdims=True)
    stable_product = product - max_prod
    probs = K.exp(stable_product) + 1
    logs = K.log(probs)
    return K.mean(logs)

loss = Lambda(loss_fn)([label_tensor, test_tensor])
model = Model(inputs=[label_tensor, test_tensor], outputs=loss)
model.predict([negative_labels, test_in])

array([ 0.08088002,  0.08088002], dtype=float32)

In [108]:
def proper_loss_fn(y_true, y_pred):
    product = y_true * y_pred
    probs = K.sigmoid(product)
    logs = K.log(probs)
    return K.mean(logs)
    
model = Model(inputs=test_tensor, outputs=test_tensor)
model.compile(optimizer='adam', loss=proper_loss_fn)
model.fit(test_in, negative_labels, batch_size=1, epochs=1)

Epoch 1/1
2/2 [==============================] - 0s - loss: -4.0809     


In [109]:
from keras.preprocessing import image

x_dir = '../sample/x/'
z_dir = '../sample/z/'

def load_images(directory, dimension, n_images, suffix):
    img_array = np.empty((n_images, dimension, dimension, 3))
    for i in range(1, n_images + 1):
        img = image.load_img(directory + str(i) + suffix, target_size=(dimension, dimension))
        img_array[i - 1] = image.img_to_array(img)
    return img_array

x_images = load_images(x_dir, 255, 100, '.x.jpg')

In [110]:
from keras.preprocessing.image import ImageDataGenerator
generator = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
generator.fit(x_images)
processed_images = generator.